<a href="https://colab.research.google.com/github/rileyzt/metastart/blob/main/StockView/StockView.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio yfinance pandas requests aiohttp

In [ ]:
import gradio as gr
import requests
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import json
from typing import Dict, List, Tuple
import asyncio
import aiohttp
from dataclasses import dataclass
import re

@dataclass
class StockAnalysis:
    """Data class to hold comprehensive stock analysis"""
    symbol: str
    company_name: str
    fundamentals: Dict
    trading_stats: Dict
    financial_health: Dict
    risk_metrics: Dict
    dividend_info: Dict
    news_links: List[str]
    reddit_links: List[str]
    twitter_links: List[str]

class StockAnalysisAgent:
    def __init__(self):
        # You'll need to add your API keys here
        self.alpha_vantage_key = "YOUR_ALPHA_VANTAGE_KEY"
        self.news_api_key = "YOUR_NEWS_API_KEY"
        self.reddit_client_id = "YOUR_REDDIT_CLIENT_ID"
        self.reddit_client_secret = "YOUR_REDDIT_CLIENT_SECRET"

    def get_stock_symbol(self, user_input: str) -> str:
        """Convert company name to stock symbol or validate symbol"""
        user_input = user_input.strip().upper()


        name_to_symbol = {
            'APPLE': 'AAPL',
            'MICROSOFT': 'MSFT',
            'GOOGLE': 'GOOGL',
            'ALPHABET': 'GOOGL',
            'AMAZON': 'AMZN',
            'TESLA': 'TSLA',
            'META': 'META',
            'FACEBOOK': 'META',
            'NETFLIX': 'NFLX',
            'NVIDIA': 'NVDA',
            'INTEL': 'INTC',
            'IBM': 'IBM',
            'ORACLE': 'ORCL',
            'SALESFORCE': 'CRM',

            'RELIANCE': 'RELIANCE.NS',
            'RELIANCE POWER': 'RPOWER.NS',
            'TATA MOTORS': 'TATAMOTORS.NS',
            'TATA': 'TATAMOTORS.NS',
            'INFOSYS': 'INFY.NS',
            'TCS': 'TCS.NS',
            'WIPRO': 'WIPRO.NS',
            'HDFC BANK': 'HDFCBANK.NS',
            'ICICI BANK': 'ICICIBANK.NS',
            'SBI': 'SBIN.NS',
            'BHARTI AIRTEL': 'BHARTIARTL.NS',
            'ITC': 'ITC.NS',
            'MARUTI': 'MARUTI.NS',
            'BAJAJ FINANCE': 'BAJFINANCE.NS'
        }


        for name, symbol in name_to_symbol.items():
            if name in user_input:
                return symbol


        if len(user_input) <= 8 and user_input.replace('.', '').isalpha():

            if not any(suffix in user_input for suffix in ['.NS', '.BO', '.L', '.TO']):

                test_symbols = [user_input, f"{user_input}.NS", f"{user_input}.BO"]
                for test_symbol in test_symbols:
                    try:
                        stock = yf.Ticker(test_symbol)
                        info = stock.info
                        if info and 'symbol' in info and info.get('regularMarketPrice') is not None:
                            return test_symbol
                    except:
                        continue
            return user_input

        return user_input

    def get_fundamentals(self, symbol: str) -> Dict:
        """Get fundamental analysis data"""
        try:
            stock = yf.Ticker(symbol)
            info = stock.info

            fundamentals = {
                'market_cap': info.get('marketCap', 'N/A'),
                'pe_ratio': info.get('trailingPE', 'N/A'),
                'forward_pe': info.get('forwardPE', 'N/A'),
                'peg_ratio': info.get('pegRatio', 'N/A'),
                'price_to_book': info.get('priceToBook', 'N/A'),
                'price_to_sales': info.get('priceToSalesTrailing12Months', 'N/A'),
                'enterprise_value': info.get('enterpriseValue', 'N/A'),
                'ev_to_revenue': info.get('enterpriseToRevenue', 'N/A'),
                'ev_to_ebitda': info.get('enterpriseToEbitda', 'N/A'),
                'profit_margin': info.get('profitMargins', 'N/A'),
                'operating_margin': info.get('operatingMargins', 'N/A'),
                'return_on_equity': info.get('returnOnEquity', 'N/A'),
                'return_on_assets': info.get('returnOnAssets', 'N/A'),
                'book_value': info.get('bookValue', 'N/A'),
                'earnings_growth': info.get('earningsGrowth', 'N/A'),
                'revenue_growth': info.get('revenueGrowth', 'N/A')
            }

            return fundamentals
        except Exception as e:
            return {'error': f'Failed to get fundamentals: {str(e)}'}

    def get_trading_stats(self, symbol: str) -> Dict:
        """Get recent trading session statistics"""
        try:
            stock = yf.Ticker(symbol)
            info = stock.info
            hist = stock.history(period="5d")

            if not hist.empty:
                latest_price = hist['Close'].iloc[-1]
                prev_close = hist['Close'].iloc[-2] if len(hist) > 1 else latest_price
                volume = hist['Volume'].iloc[-1]
                avg_volume = hist['Volume'].mean()
            else:
                latest_price = info.get('currentPrice', 'N/A')
                prev_close = info.get('previousClose', 'N/A')
                volume = info.get('volume', 'N/A')
                avg_volume = info.get('averageVolume', 'N/A')

            trading_stats = {
                'current_price': latest_price,
                'previous_close': prev_close,
                'day_change': (latest_price - prev_close) if isinstance(latest_price, (int, float)) and isinstance(prev_close, (int, float)) else 'N/A',
                'day_change_percent': ((latest_price - prev_close) / prev_close * 100) if isinstance(latest_price, (int, float)) and isinstance(prev_close, (int, float)) else 'N/A',
                'volume': volume,
                'avg_volume': avg_volume,
                'day_high': info.get('dayHigh', 'N/A'),
                'day_low': info.get('dayLow', 'N/A'),
                '52_week_high': info.get('fiftyTwoWeekHigh', 'N/A'),
                '52_week_low': info.get('fiftyTwoWeekLow', 'N/A'),
                'beta': info.get('beta', 'N/A'),
                'shares_outstanding': info.get('sharesOutstanding', 'N/A')
            }

            return trading_stats
        except Exception as e:
            return {'error': f'Failed to get trading stats: {str(e)}'}

    def get_financial_health(self, symbol: str) -> Dict:
        """Get financial health metrics"""
        try:
            stock = yf.Ticker(symbol)
            info = stock.info

            financial_health = {
                'total_cash': info.get('totalCash', 'N/A'),
                'total_debt': info.get('totalDebt', 'N/A'),
                'debt_to_equity': info.get('debtToEquity', 'N/A'),
                'current_ratio': info.get('currentRatio', 'N/A'),
                'quick_ratio': info.get('quickRatio', 'N/A'),
                'cash_per_share': info.get('totalCashPerShare', 'N/A'),
                'revenue_ttm': info.get('totalRevenue', 'N/A'),
                'gross_profit': info.get('grossProfits', 'N/A'),
                'ebitda': info.get('ebitda', 'N/A'),
                'net_income': info.get('netIncomeToCommon', 'N/A'),
                'free_cash_flow': info.get('freeCashflow', 'N/A'),
                'operating_cash_flow': info.get('operatingCashflow', 'N/A')
            }

            return financial_health
        except Exception as e:
            return {'error': f'Failed to get financial health: {str(e)}'}

    def get_risk_metrics(self, symbol: str) -> Dict:
        """Calculate risk management metrics"""
        try:
            stock = yf.Ticker(symbol)
            hist = stock.history(period="1y")
            info = stock.info

            if not hist.empty:
                returns = hist['Close'].pct_change().dropna()
                volatility = returns.std() * (252 ** 0.5)  # Annualized volatility
                max_drawdown = self.calculate_max_drawdown(hist['Close'])
            else:
                volatility = 'N/A'
                max_drawdown = 'N/A'

            risk_metrics = {
                'beta': info.get('beta', 'N/A'),
                'volatility_1y': volatility,
                'max_drawdown_1y': max_drawdown,
                'analyst_rating': info.get('recommendationKey', 'N/A'),
                'target_high_price': info.get('targetHighPrice', 'N/A'),
                'target_low_price': info.get('targetLowPrice', 'N/A'),
                'target_mean_price': info.get('targetMeanPrice', 'N/A'),
                'number_of_analysts': info.get('numberOfAnalystOpinions', 'N/A')
            }

            return risk_metrics
        except Exception as e:
            return {'error': f'Failed to get risk metrics: {str(e)}'}

    def calculate_max_drawdown(self, prices) -> float:
        """Calculate maximum drawdown"""
        try:
            peak = prices.expanding().max()
            drawdown = (prices - peak) / peak
            return drawdown.min()
        except:
            return 'N/A'

    def get_dividend_info(self, symbol: str) -> Dict:
        """Get dividend policy information"""
        try:
            stock = yf.Ticker(symbol)
            info = stock.info

            dividend_info = {
                'dividend_yield': info.get('dividendYield', 'N/A'),
                'dividend_rate': info.get('dividendRate', 'N/A'),
                'ex_dividend_date': info.get('exDividendDate', 'N/A'),
                'payout_ratio': info.get('payoutRatio', 'N/A'),
                'five_year_avg_yield': info.get('fiveYearAvgDividendYield', 'N/A'),
                'dividend_growth_rate': 'N/A'
            }

            return dividend_info
        except Exception as e:
            return {'error': f'Failed to get dividend info: {str(e)}'}

    def search_news_articles(self, symbol: str, company_name: str) -> List[str]:
        """Search for latest news articles about the stock"""
        try:
            # Using yfinance news (free alternative)
            stock = yf.Ticker(symbol)
            news = stock.news

            news_links = []
            for article in news[:5]:  # Get top 5 articles
                if 'link' in article:
                    news_links.append(article['link'])

            # Alternative: You can use News API if you have a key
            # url = f"https://newsapi.org/v2/everything?q={symbol}&apiKey={self.news_api_key}&sortBy=publishedAt&pageSize=5"

            return news_links
        except Exception as e:
            return [f"Error fetching news: {str(e)}"]

    def search_reddit_discussions(self, symbol: str) -> List[str]:
        """Search for Reddit discussions about the stock"""
        try:
            # Reddit search URLs (doesn't require API key)
            reddit_links = [
                f"https://www.reddit.com/r/investing/search/?q={symbol}&restrict_sr=1&sort=new",
                f"https://www.reddit.com/r/stocks/search/?q={symbol}&restrict_sr=1&sort=new",
                f"https://www.reddit.com/r/SecurityAnalysis/search/?q={symbol}&restrict_sr=1&sort=new",
                f"https://www.reddit.com/r/ValueInvesting/search/?q={symbol}&restrict_sr=1&sort=new",
                f"https://www.reddit.com/r/StockMarket/search/?q={symbol}&restrict_sr=1&sort=new"
            ]

            return reddit_links
        except Exception as e:
            return [f"Error fetching Reddit links: {str(e)}"]

    def search_twitter_mentions(self, symbol: str) -> List[str]:
        """Search for Twitter mentions about the stock"""
        try:
            # Twitter search URLs (public search)
            twitter_links = [
                f"https://twitter.com/search?q=${symbol}&src=typed_query&f=live",
                f"https://twitter.com/search?q={symbol}%20stock&src=typed_query&f=live",
                f"https://twitter.com/search?q={symbol}%20earnings&src=typed_query&f=live",
                f"https://twitter.com/search?q={symbol}%20analysis&src=typed_query&f=live"
            ]

            return twitter_links
        except Exception as e:
            return [f"Error fetching Twitter links: {str(e)}"]

    def format_analysis_report(self, analysis: StockAnalysis) -> str:
        """Format the complete analysis into a readable report"""

        def format_value(value, is_currency=True, decimal_places=2):
            if value == 'N/A' or value is None or value == '':
                return 'N/A'

            try:

                if isinstance(value, str):

                    clean_value = re.sub(r'[^\d.-]', '', str(value))
                    if clean_value and clean_value != '-':
                        value = float(clean_value)
                    else:
                        return 'N/A'

                if isinstance(value, (int, float)):
                    if is_currency:
                        if abs(value) > 1000000000:
                            return f"${value/1000000000:.{decimal_places}f}B"
                        elif abs(value) > 1000000:
                            return f"${value/1000000:.{decimal_places}f}M"
                        elif abs(value) > 1000:
                            return f"${value/1000:.{decimal_places}f}K"
                        else:
                            return f"${value:.{decimal_places}f}"
                    else:
                        return f"{value:.{decimal_places}f}"
                else:
                    return str(value)
            except (ValueError, TypeError):
                return str(value) if value else 'N/A'

        report = f"""
# 📊 Stock Analysis Report: {analysis.company_name} ({analysis.symbol})

## 🔍 Fundamental Analysis
- **Market Cap**: {format_value(analysis.fundamentals.get('market_cap'))}
- **P/E Ratio**: {format_value(analysis.fundamentals.get('pe_ratio'), False)}
- **Forward P/E**: {format_value(analysis.fundamentals.get('forward_pe'), False)}
- **PEG Ratio**: {format_value(analysis.fundamentals.get('peg_ratio'), False)}
- **Price-to-Book**: {format_value(analysis.fundamentals.get('price_to_book'), False)}
- **Price-to-Sales**: {format_value(analysis.fundamentals.get('price_to_sales'), False)}
- **Enterprise Value**: {format_value(analysis.fundamentals.get('enterprise_value'))}
- **EV/Revenue**: {format_value(analysis.fundamentals.get('ev_to_revenue'), False)}
- **EV/EBITDA**: {format_value(analysis.fundamentals.get('ev_to_ebitda'), False)}

## 📈 Trading Statistics
- **Current Price**: {format_value(analysis.trading_stats.get('current_price'))}
- **Previous Close**: {format_value(analysis.trading_stats.get('previous_close'))}
- **Day Change**: {format_value(analysis.trading_stats.get('day_change'))} ({format_value(analysis.trading_stats.get('day_change_percent'), False)}%)
- **Volume**: {format_value(analysis.trading_stats.get('volume'), False, 0)}
- **Average Volume**: {format_value(analysis.trading_stats.get('avg_volume'), False, 0)}
- **Day Range**: {format_value(analysis.trading_stats.get('day_low'))} - {format_value(analysis.trading_stats.get('day_high'))}
- **52-Week Range**: {format_value(analysis.trading_stats.get('52_week_low'))} - {format_value(analysis.trading_stats.get('52_week_high'))}
- **Beta**: {format_value(analysis.trading_stats.get('beta'), False)}

## 💰 Financial Health
- **Total Cash**: {format_value(analysis.financial_health.get('total_cash'))}
- **Total Debt**: {format_value(analysis.financial_health.get('total_debt'))}
- **Debt-to-Equity**: {format_value(analysis.financial_health.get('debt_to_equity'), False)}
- **Current Ratio**: {format_value(analysis.financial_health.get('current_ratio'), False)}
- **Quick Ratio**: {format_value(analysis.financial_health.get('quick_ratio'), False)}
- **Revenue (TTM)**: {format_value(analysis.financial_health.get('revenue_ttm'))}
- **Gross Profit**: {format_value(analysis.financial_health.get('gross_profit'))}
- **EBITDA**: {format_value(analysis.financial_health.get('ebitda'))}
- **Free Cash Flow**: {format_value(analysis.financial_health.get('free_cash_flow'))}

## ⚠️ Risk Management
- **Beta**: {format_value(analysis.risk_metrics.get('beta'), False)}
- **1-Year Volatility**: {format_value(analysis.risk_metrics.get('volatility_1y'), False)}%
- **Max Drawdown (1Y)**: {format_value(analysis.risk_metrics.get('max_drawdown_1y'), False)}%
- **Analyst Rating**: {analysis.risk_metrics.get('analyst_rating', 'N/A')}
- **Price Targets**: {format_value(analysis.risk_metrics.get('target_low_price'))} - {format_value(analysis.risk_metrics.get('target_high_price'))} (Mean: {format_value(analysis.risk_metrics.get('target_mean_price'))})

## 💎 Dividend Policy
- **Dividend Yield**: {format_value(analysis.dividend_info.get('dividend_yield'), False)}%
- **Dividend Rate**: {format_value(analysis.dividend_info.get('dividend_rate'))}
- **Ex-Dividend Date**: {analysis.dividend_info.get('ex_dividend_date', 'N/A')}
- **Payout Ratio**: {format_value(analysis.dividend_info.get('payout_ratio'), False)}%
- **5-Year Avg Yield**: {format_value(analysis.dividend_info.get('five_year_avg_yield'), False)}%

## 📰 Latest News Articles
"""

        for i, link in enumerate(analysis.news_links[:5], 1):
            report += f"{i}. {link}\n"

        report += f"""
## 💬 Reddit Discussions
"""

        for i, link in enumerate(analysis.reddit_links[:5], 1):
            report += f"{i}. {link}\n"

        report += f"""
## 🐦 Twitter Mentions
"""

        for i, link in enumerate(analysis.twitter_links[:4], 1):
            report += f"{i}. {link}\n"

        report += f"""
---
*Analysis generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""

        return report

    def analyze_stock(self, user_input: str, progress=gr.Progress()) -> str:
        """Main function to perform comprehensive stock analysis"""

        if not user_input.strip():
            return "❌ Please enter a stock symbol or company name."

        progress(0.1, desc="Resolving stock symbol...")
        symbol = self.get_stock_symbol(user_input)

        try:

            stock = yf.Ticker(symbol)
            info = stock.info


            if not info or len(info) < 5:
                return f"❌ Could not find data for '{user_input}'. Please check the symbol.\n\n**Suggestions:**\n- For Indian stocks, try adding .NS (e.g., RELIANCE.NS)\n- For US stocks, use the ticker symbol (e.g., AAPL for Apple)\n- Check if the company is publicly traded"

            company_name = info.get('longName', info.get('shortName', symbol))

            progress(0.2, desc="Gathering fundamental data...")
            fundamentals = self.get_fundamentals(symbol)

            progress(0.4, desc="Analyzing trading statistics...")
            trading_stats = self.get_trading_stats(symbol)

            progress(0.6, desc="Evaluating financial health...")
            financial_health = self.get_financial_health(symbol)

            progress(0.7, desc="Calculating risk metrics...")
            risk_metrics = self.get_risk_metrics(symbol)

            progress(0.8, desc="Getting dividend information...")
            dividend_info = self.get_dividend_info(symbol)

            progress(0.85, desc="Searching for news articles...")
            news_links = self.search_news_articles(symbol, company_name)

            progress(0.9, desc="Finding Reddit discussions...")
            reddit_links = self.search_reddit_discussions(symbol)

            progress(0.95, desc="Gathering Twitter mentions...")
            twitter_links = self.search_twitter_mentions(symbol)

            progress(1.0, desc="Generating report...")

            # Create analysis object
            analysis = StockAnalysis(
                symbol=symbol,
                company_name=company_name,
                fundamentals=fundamentals,
                trading_stats=trading_stats,
                financial_health=financial_health,
                risk_metrics=risk_metrics,
                dividend_info=dividend_info,
                news_links=news_links,
                reddit_links=reddit_links,
                twitter_links=twitter_links
            )

            return self.format_analysis_report(analysis)

        except Exception as e:
            return f"❌ Error analyzing '{user_input}': {str(e)}\n\n**Troubleshooting:**\n- Verify the stock symbol is correct\n- For Indian stocks, try adding .NS (e.g., RELIANCE.NS)\n- For other international stocks, try adding country suffix (.L for London, .TO for Toronto)\n- Check your internet connection"

def create_gradio_interface():
    """Create the Gradio interface"""

    agent = StockAnalysisAgent()

    # Custom CSS for better styling
    css = """
    .gradio-container {
        max-width: 1200px !important;
    }
    .main-header {
        text-align: center;
        color: #2E8B57;
        margin-bottom: 20px;
    }
    .analysis-output {
        font-family: 'Monaco', monospace;
        line-height: 1.6;
    }
    """

    with gr.Blocks(css=css, title="Stock Analysis AI Agent") as demo:
        gr.Markdown("""
        # 🤖 Stock Analysis AI Agent

        **Enter any stock symbol (e.g., AAPL, MSFT) or company name (e.g., Apple, Microsoft) to get comprehensive analysis**

        This AI agent will analyze:
        - 📊 Fundamental metrics (P/E, PEG, Market Cap, etc.)
        - 📈 Trading statistics and price movements
        - 💰 Financial health and cash flow
        - ⚠️ Risk management metrics
        - 💎 Dividend policy and yield
        - 📰 Latest news articles
        - 💬 Reddit discussions
        - 🐦 Twitter mentions
        """, elem_classes="main-header")

        with gr.Row():
            with gr.Column(scale=3):
                stock_input = gr.Textbox(
                    label="Stock Symbol or Company Name",
                    placeholder="Enter AAPL, Apple, MSFT, Microsoft, etc.",
                    value="AAPL"
                )

            with gr.Column(scale=1):
                analyze_btn = gr.Button("🔍 Analyze Stock", variant="primary", scale=1)

        with gr.Row():
            output = gr.Markdown(
                label="Analysis Report",
                elem_classes="analysis-output"
            )


        analyze_btn.click(
            fn=agent.analyze_stock,
            inputs=[stock_input],
            outputs=[output],
            show_progress=True
        )

        stock_input.submit(
            fn=agent.analyze_stock,
            inputs=[stock_input],
            outputs=[output],
            show_progress=True
        )

        # Example buttons
        gr.Markdown("### 🎯 Quick Examples:")
        with gr.Row():
            example_buttons = []
            examples = [
                ("Apple (AAPL)", "AAPL"),
                ("Microsoft (MSFT)", "MSFT"),
                ("Tesla (TSLA)", "TSLA"),
                ("NVIDIA (NVDA)", "NVDA"),
                ("Reliance (RELIANCE.NS)", "RELIANCE.NS"),
                ("Tata Motors (TATAMOTORS.NS)", "TATAMOTORS.NS")
            ]

            for label, symbol in examples:
                btn = gr.Button(label, variant="secondary", scale=1)
                btn.click(
                    fn=lambda s=symbol: s,
                    outputs=[stock_input]
                )

        gr.Markdown("""
        ### 📝 Notes:
        - Analysis uses real-time data from Yahoo Finance
        - News links are fetched from latest financial news sources
        - Reddit and Twitter links provide community sentiment
        - All data is for informational purposes only - not investment advice

        ### 🔧 Setup Requirements:
        ```bash
        pip install gradio yfinance pandas requests aiohttp
        ```
        """)

    return demo

if __name__ == "__main__":
    # Create and launch the interface
    demo = create_gradio_interface()
    demo.launch(
        server_name="0.0.0.0",  # Allow external access
        server_port=7860,       # Default Gradio port
        share=True,             # Create public link
        debug=True              # Enable debug mode
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1a663bc2a4f0b2c930.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://1a663bc2a4f0b2c930.gradio.live


In [ ]:
demo = create_gradio_interface()
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://59a4774ac8db39e8c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:$HCL: possibly delisted; no price data found  (period=5d)
ERROR:yfinance:$HCL: possibly delisted; no price data found  (period=1y)
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:$ADANIPOWER: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ADANIPOWER: possibly delisted; no price data found  (period=1y